# Demonstration Experiments with Precomputation and Caching - MSMARCO v2

This notebook contains example experiments demonstrating the efficiency benefit of using precomputation within pt.Experiment(), and the use of a ScorerCache around an expensive cross-encoder.

This notebook was used to produce the timings reported in ``On Precomputation and Caching in Information Retrieval Experiments with Pipeline Architectures'', published in Second International Workshop on Open Web Search (WOWS 2025).

```bibtex
@{macdonald2025wows,
 author = {Craig Macdonald and Sean MacAvaney},
 title = {On Precomputation and Caching in Information Retrieval Experiments with Pipeline Architectures},
 booktitle = {Proceedings of the Second International Workshop on Open Web Search (WOWS 2025)},
 year = 2025,
}
```

Experiments are conducted on the MSMARCO v2 passage corpus, using the 53 queries of the TREC 2021 Deep Learning track,

In [1]:
%pip install -q python-terrier pyterrier_caching pyterrier_t5

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyterrier as pt

Load our BM25 retrieval index

In [12]:
index = pt.IndexFactory.of(pt.get_dataset('msmarcov2_passage').get_index('terrier_stemmed'), memory=['meta'])
bm25 = pt.terrier.Retriever(index, wmodel='BM25', verbose=True) >> pt.text.get_text(pt.get_dataset('irds:msmarco-passage-v2'), ['text'], verbose=True)

In [13]:
bm25.search("chemical reactions")

IRDSTextLoader: 100%|██████████| 1000/1000 [00:00<00:00, 1614.97d/s]


,qid,docid,docno,rank,score,query,text
0,1,8399388,msmarco_passage_04_171124771,0,28.421805,chemical reactions,Not ALL reactions are chemicals. When chemical...
1,1,65097464,msmarco_passage_32_479263716,1,28.092676,chemical reactions,How is the speed of a chemical reaction relate...
2,1,8199479,msmarco_passage_04_86253150,2,28.080419,chemical reactions,The chemical or chemicals formed in a chemical...
3,1,87279763,msmarco_passage_43_554702663,3,27.908001,chemical reactions,Necessity in Chemical Reactions: Reactants are...
4,1,4792937,msmarco_passage_02_344129610,4,27.893566,chemical reactions,Chemical Reactions Types Worksheet Beautiful C...
...,...,...,...,...,...,...,...
995,1,49259151,msmarco_passage_24_547713764,995,24.766633,chemical reactions,A simple example of a chemical reaction would ...
996,1,52499541,msmarco_passage_26_217980777,996,24.766633,chemical reactions,Reaction: ↑ A chemical reaction occurs when at...
997,1,61395339,msmarco_passage_30_611194941,997,24.766633,chemical reactions,endothermic. chemical reaction absorbs more en...
998,1,61774696,msmarco_passage_30_778898578,998,24.766633,chemical reactions,The mass of 1 mole of a substance. Products. T...


In [14]:
from pyterrier_t5 import MonoT5ReRanker, DuoT5ReRanker
monoT5 = MonoT5ReRanker() # loads castorini/monot5-base-msmarco by default
duoT5 = DuoT5ReRanker() # loads castorini/duot5-base-msmarco by default

/opt/miniconda3/envs/rag/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
dataset = pt.get_dataset("irds:msmarco-passage-v2/trec-dl-2021/judged")

In [ ]:
!rm -rf monoT5.cache.v2

In [16]:
from pyterrier.measures import *
def one(): # no precomputation, no caching
    return pt.Experiment(
        [bm25 % k >> monoT5 % 10 >> duoT5 for k in [20, 50, 100, 200]],
        dataset.get_topics(),
        dataset.get_qrels(),
        [nDCG@10],
        precompute_prefix=False # <---- disable precomputation 
    )

def two(): # precomputation, no caching
    return pt.Experiment(
        [bm25 % k >> monoT5 % 10 >> duoT5 for k in [20, 50, 100, 200]],
        dataset.get_topics(),
        dataset.get_qrels(),
        [nDCG@10],
        precompute_prefix=True # <---- enable precomputation 
    )

def three_and_four(): # precomputation, caching
    from pyterrier_caching import ScorerCache
    cached_scorer = ScorerCache('monoT5.cache.v2', monoT5)
    return pt.Experiment(
        [bm25 % k >> cached_scorer % 10 >> duoT5 for k in [20, 50, 100, 200]],
        dataset.get_topics(),
        dataset.get_qrels(),
        [nDCG@10],
        precompute_prefix=True # <---- enable precomputation 
    )

In [18]:
%time one()

duoT5: 100%|██████████| 53/53 [00:32<00:00,  1.65queries/s]

CPU times: user 5min 47s, sys: 5.64 s, total: 5min 53s
Wall time: 5min 48s


,name,nDCG@10
0,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.567805
1,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.610255
2,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.636022
3,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.647122


In [17]:
%time two()

Precomputing results of 53 topics on shared pipeline component (TerrierRetr(BM25) >> <pyterrier.datasets.IRDSTextLoader object at 0x7f9fc8124250>)
duoT5: 100%|██████████| 53/53 [00:26<00:00,  1.98queries/s]

CPU times: user 4min 10s, sys: 2.95 s, total: 4min 13s
Wall time: 4min 12s


,name,nDCG@10
0,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.567805
1,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.610255
2,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.636022
3,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.647122


In [19]:
%time three_and_four()

Precomputing results of 53 topics on shared pipeline component (TerrierRetr(BM25) >> <pyterrier.datasets.IRDSTextLoader object at 0x7f9fc8124250>)
duoT5: 100%|██████████| 53/53 [00:27<00:00,  1.95queries/s]

CPU times: user 3min 24s, sys: 2.87 s, total: 3min 27s
Wall time: 3min 27s


,name,nDCG@10
0,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.567805
1,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.610002
2,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.634912
3,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.644595


In [20]:
%time three_and_four()

Precomputing results of 53 topics on shared pipeline component (TerrierRetr(BM25) >> <pyterrier.datasets.IRDSTextLoader object at 0x7f9fc8124250>)
duoT5: 100%|██████████| 53/53 [00:27<00:00,  1.95queries/s]

CPU times: user 2min 23s, sys: 2.12 s, total: 2min 25s
Wall time: 2min 25s


,name,nDCG@10
0,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.567805
1,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.610002
2,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.634912
3,(TerrierRetr(BM25) >> <pyterrier.datasets.IRDS...,0.644595


NB: For `three_and_four()`, there are very minor changes in effectiveness, at the 5th decimal place, compared to `one()` and `two()`. This is due to changes in GPU batching when scoring MonoT5. The scores should usually be the same when the order of the data onto the GPU is the same, but differences can be observed (e.g.) a different batch due to some of the values being cached from a prior round.